In [5]:
import pandas as pd

result1 = pd.read_csv('result csv/chatbot_responses_auto.csv')
result2 = pd.read_csv('result csv/chatbot_responses_trained.csv')
result3 = pd.read_csv('result csv/chatbot_responses_bayesian_previous.csv')

In [12]:
result1.head()

,input_question,answer
0,화상연고도 처방이 되나요?,저희 병원에서 사용되는 의약품은 아래층에 행복 약국에서 구매가 가능합니다.
1,오늘 두시 삼십분에 예약했어요. 언제 들어가나요?,어서 오십시오. 성함과 연락처를 앞에 적어주시면 확인 후 접수 도와드리겠습니다.
2,예약한 시간보다 조금 늦었어요. 다시 접수해야 하나요?,대기하시기 전에 접수증 작성 부탁드립니다.
3,"피가 계속 나는데, 저 죽는 걸까요?",긴급상황이라면 응급실로 이동하셔서 진료 보실 수 있습니다. 응급실 건물로 가주시면 ...
4,접수 재등록해도 되나요?,마감 정산이 필요하여 지금 수납해 주시면 원활한 처리가 가능합니다.


In [24]:
result2_answer = result2[['best_answer']]
result2_answer.head()
# 열 이름 변경
result2_answer = result2_answer.rename(columns={
    'best_answer': 'trained_model'
})

In [20]:
result3_answer = result3[['best_answer']]
result3_answer.head()

,best_answer
0,우측 복도 끝에 위치해 있습니다. 심리평가실까지 안내해 드리겠습니다.
1,긴급상황이라면 응급실로 이동하셔서 진료 보실 수 있습니다. 응급실 건물로 가주시면 ...
2,반갑습니다. 1층 통합 창구에서 번호표를 뽑고 기다려 주시면 안내 도와드리겠습니다.
3,1층 창구에서 진료 접수를 할 수 있습니다. 저를 따라오시면 안내해 드리겠습니다.
4,현재 고객님 앞의 대기자가 0명이기 때문에 즉시 진료 보실 수 있습니다.


In [25]:
# 두 DataFrame을 열 방향으로 합치기
combined_df = pd.concat([result1,result2_answer.reset_index(drop=True), result3_answer.reset_index(drop=True)], axis=1)

In [26]:
combined_df.head()

,input_question,answer,trained_model,best_answer
0,화상연고도 처방이 되나요?,저희 병원에서 사용되는 의약품은 아래층에 행복 약국에서 구매가 가능합니다.,진료실 옆 대기석에 등받이 있는 의자가 있습니다. 안내해 드리겠습니다.,우측 복도 끝에 위치해 있습니다. 심리평가실까지 안내해 드리겠습니다.
1,오늘 두시 삼십분에 예약했어요. 언제 들어가나요?,어서 오십시오. 성함과 연락처를 앞에 적어주시면 확인 후 접수 도와드리겠습니다.,죄송합니다. 앞서 예약하신 손님이 있어서 예약시간대로 기다려주셔야 합니다.,긴급상황이라면 응급실로 이동하셔서 진료 보실 수 있습니다. 응급실 건물로 가주시면 ...
2,예약한 시간보다 조금 늦었어요. 다시 접수해야 하나요?,대기하시기 전에 접수증 작성 부탁드립니다.,앞선 예약이 없어서 가능합니다. 예약시간 한 시간 미뤄 드리겠습니다.,반갑습니다. 1층 통합 창구에서 번호표를 뽑고 기다려 주시면 안내 도와드리겠습니다.
3,"피가 계속 나는데, 저 죽는 걸까요?",긴급상황이라면 응급실로 이동하셔서 진료 보실 수 있습니다. 응급실 건물로 가주시면 ...,피부 사정에 따라 달라서 말씀드리기 어렵습니다. 증상은 사람마다 다릅니다.,1층 창구에서 진료 접수를 할 수 있습니다. 저를 따라오시면 안내해 드리겠습니다.
4,접수 재등록해도 되나요?,마감 정산이 필요하여 지금 수납해 주시면 원활한 처리가 가능합니다.,제가 안내해 드리겠습니다. 이쪽으로 따라오시면 됩니다.,현재 고객님 앞의 대기자가 0명이기 때문에 즉시 진료 보실 수 있습니다.


In [30]:
# 열 이름 변경
df_renamed = combined_df.rename(columns={
    'input_question': 'input_question',
    'answer': 'automodel',
    'trained_model': 'trained_model',
    'best_answer': 'bayesian_model'
})
df_renamed.to_excel('combined_responses.xlsx', index=False)
df_renamed.to_csv('result csv/final_responses.csv', index=False)

In [31]:
df_renamed

,input_question,automodel,trained_model,bayesian_model
0,화상연고도 처방이 되나요?,저희 병원에서 사용되는 의약품은 아래층에 행복 약국에서 구매가 가능합니다.,진료실 옆 대기석에 등받이 있는 의자가 있습니다. 안내해 드리겠습니다.,우측 복도 끝에 위치해 있습니다. 심리평가실까지 안내해 드리겠습니다.
1,오늘 두시 삼십분에 예약했어요. 언제 들어가나요?,어서 오십시오. 성함과 연락처를 앞에 적어주시면 확인 후 접수 도와드리겠습니다.,죄송합니다. 앞서 예약하신 손님이 있어서 예약시간대로 기다려주셔야 합니다.,긴급상황이라면 응급실로 이동하셔서 진료 보실 수 있습니다. 응급실 건물로 가주시면 ...
2,예약한 시간보다 조금 늦었어요. 다시 접수해야 하나요?,대기하시기 전에 접수증 작성 부탁드립니다.,앞선 예약이 없어서 가능합니다. 예약시간 한 시간 미뤄 드리겠습니다.,반갑습니다. 1층 통합 창구에서 번호표를 뽑고 기다려 주시면 안내 도와드리겠습니다.
3,"피가 계속 나는데, 저 죽는 걸까요?",긴급상황이라면 응급실로 이동하셔서 진료 보실 수 있습니다. 응급실 건물로 가주시면 ...,피부 사정에 따라 달라서 말씀드리기 어렵습니다. 증상은 사람마다 다릅니다.,1층 창구에서 진료 접수를 할 수 있습니다. 저를 따라오시면 안내해 드리겠습니다.
4,접수 재등록해도 되나요?,마감 정산이 필요하여 지금 수납해 주시면 원활한 처리가 가능합니다.,제가 안내해 드리겠습니다. 이쪽으로 따라오시면 됩니다.,현재 고객님 앞의 대기자가 0명이기 때문에 즉시 진료 보실 수 있습니다.
5,오래된 상처 흉터도 없어질까요?,미용 목적의 수술의 경우 진단서 발급이 따로 어렵습니다.,잠시만 기다려 주십시오. 우선적으로 내과를 접수하시면 되겠습니다.,정밀 검사 후에 감마 상태를 보고 결정될 것 같습니다.
6,여기 한의원도 같이 하나요? 침을 맡고 싶어요.,정수기는 진료 대기실 앞쪽에 있습니다. 감사합니다.,수액 클리닉 실은 복도 끝에 있습니다. 수액 클리닉 실로같이 동행하겠습니다.,맞습니다. 증상 상담 시 자세한 증상을 알려주시길 부탁드립니다.
7,목 디스크 때문에 잠을 잘 수 없어요. 어떤 치료를 받을 수 있나요?,진료는 진료실에서 전문가와 함께 상담 부탁드립니다.,"흉부외과를 먼저 진료 보신 후, 담당 의사 선생님과 자세히 상담하시길 추천드립니다.",상담 접수를 진행하였습니다. 접수 완료 문자를 등록된 번호로 전송합니다.
8,"탈모 초기 인 것 같아요. 머리카락이 자꾸 빠지는데, 이것도 진료를 볼 수 있나요?",체온 측정을 데스크에서 도와드리겠습니다. 데스크로 이동해 주시기 바랍니다.,검사 접수 도와드리겠습니다. 지금 드리는 문진표를 작성해 주십시오.,의사선생님이 오시면 환자분의 증상에 대한 자세한 상담을 받으실 수 있습니다.
9,"저는 아니고, 제 아이가 열이 자꾸 오르다 말다 해요. 소아과도 진료 하나요?",평범한 감기일 수도 있으니까 너무 걱정 마십시오. 열이 나는 것 이외에 증상 있으십니까?,평범한 감기일 수도 있으니까 너무 걱정 마십시오. 열이 나는 것 이외에 증상 있으십니까?,상담 접수를 진행하였습니다. 접수 완료 문자를 등록된 번호로 전송합니다.
